In [2]:
import pandas as pd
import numpy as np

from influxdb_client import InfluxDBClient, Point, Dialect

import re
import time
import datetime

import warnings
from influxdb_client.client.warnings import MissingPivotFunction

import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
import matplotlib.colors as colors

import pandasql as ps
import sqlite3

import csv

pd.set_option('display.max_rows', 500)

In [3]:
def _parse_line(line):

    rx_dict = {
    'token': re.compile(r'var token = "(?P<token>.*)"\n'),
    'url': re.compile(r'var url = "(?P<url>.*)"\n'),
    'org': re.compile(r'var org = "(?P<org>.*)"\n'),
    'bucket': re.compile(r'var bucket = "(?P<bucket>.*)"\n'),
    }   

    """
    Do a regex search against all defined regexes and
    return the key and match result of the first matching regex

    """
    for key, rx in rx_dict.items():
        match = rx.search(line)
        if match:
            return key, match
    # if there are no matches
    return None, None


filepath = '/root/flexi-pipe/config.go'
# open the file and read through it line by line
with open(filepath, 'r') as file_object:
    line = file_object.readline()
    while line:
        # at each line check for a match with a regex
        key, match = _parse_line(line)

        if key == 'token':
            token = match.group('token')
        elif key == 'url':
            url = match.group('url')
        elif key == 'org':
            org = match.group('org')
        elif key == 'bucket':
            bucket = match.group('bucket')
        
        line = file_object.readline()
# url="http://192.168.20.58:8086"
url = "http://localhost:8086"


In [8]:
start_time = 1692978196
end_time = 1693129062

filepath = "../experiments.csv"

# Retrieve experiments data from csv
data = pd.read_csv(filepath, header=None)
df = pd.DataFrame(data)

#Rename columns
experiments = df.rename(columns={0: "start", 1: "end", 2: "topology", 3: "runtime", 4: "parameter", 5: "d", 6: "dlo", 7: "dhi", 8: "dscore", 9: "dlazy", 10: "dout", 11: "gossipFactor", 12: "initialDelay", 13: "interval"}, errors='raise')

#Correct timestamp
experiments["start"] = experiments["start"].str.slice(0, 27)
experiments["end"] = experiments["end"].str.slice(0, 27)

#String to timestamp
# experiments['startUnix'] = pd.to_datetime(experiments["start"],format="%Y-%m-%d %H:%M:%S.%f").astype('int64') / 10**9
# experiments['endUnix'] = pd.to_datetime(experiments["end"],format="%Y-%m-%d %H:%M:%S.%f").astype('int64') / 10**9
experiments['startUnix'] = pd.to_datetime(experiments["start"],format="mixed").astype('int64') / 10**9
experiments['endUnix'] = pd.to_datetime(experiments["end"],format="mixed").astype('int64') / 10**9

experiments['startUnix'] = pd.to_timedelta(experiments['startUnix'], unit='s').dt.total_seconds().astype(int)#.astype(str)
experiments['endUnix'] = pd.to_timedelta(experiments['endUnix'], unit='s').dt.total_seconds().astype(int)#.astype(str)

#Drop fields we don't mneed for the moment
exp = experiments.drop(columns=["runtime", "initialDelay"]).sort_values(by=["start"])

#Get times for different intervals
intervals = exp["interval"].drop_duplicates().sort_values().reset_index(drop=True)
# intervals.head(10)

# expTime = exp[exp['startUnix'].astype(int).between(start_time, end_time)]
# expTime['experiment'] = expTime.index
expTime = exp.reset_index().rename({'index':'experiment'}, axis = 'columns')

expTime.tail(500)

,experiment,start,end,topology,parameter,d,dlo,dhi,dscore,dlazy,dout,gossipFactor,interval,startUnix,endUnix
0,0,2023-08-21 12:48:22.3741823,2023-08-21 13:18:22.3952804,unl,reference,8,6,12,4,8,2,0.25,1.00,1692622102,1692623902
1,1,2023-08-21 13:21:55.6321388,2023-08-21 13:51:55.6334369,unl,reference,8,6,12,4,8,2,0.25,1.00,1692624115,1692625915
2,2,2023-08-21 13:55:28.9783578,2023-08-21 14:25:28.9820206,unl,reference,8,6,12,4,8,2,0.25,1.00,1692626128,1692627928
3,3,2023-08-21 14:29:02.5866046,2023-08-21 14:59:02.6871797,unl,interval,8,6,12,4,8,2,0.25,0.50,1692628142,1692629942
4,4,2023-08-21 15:02:35.9708968,2023-08-21 15:32:36.0272486,unl,interval,8,6,12,4,8,2,0.25,0.50,1692630155,1692631956
5,5,2023-08-21 15:36:09.3380555,2023-08-21 16:06:09.3391894,unl,interval,8,6,12,4,8,2,0.25,0.50,1692632169,1692633969
6,6,2023-08-21 16:09:43.1463718,2023-08-21 16:39:43.1814633,unl,interval,8,6,12,4,8,2,0.25,30.00,1692634183,1692635983
7,7,2023-08-21 16:43:16.7045785,2023-08-21 17:13:16.7785160,unl,interval,8,6,12,4,8,2,0.25,30.00,1692636196,1692637996
8,8,2023-08-21 17:16:50.1424767,2023-08-21 17:46:50.1924402,unl,interval,8,6,12,4,8,2,0.25,30.00,1692638210,1692640010
9,9,2023-08-21 17:50:23.0310174,2023-08-21 18:20:23.0333186,unl,interval,8,6,12,4,8,2,0.25,3.00,1692640223,1692642023


In [4]:
experiments = expTime

In [5]:
ref = experiments.loc[experiments["parameter"] == "reference"]
start_reference = ref["startUnix"].min().astype(int)
end_reference = ref["endUnix"].max().astype(int)

print(start_reference,end_reference)

1692978196 1692984023


In [6]:
def from_influx(url, token, org, measurement, start_time, end_time,grouping_key):
    client = InfluxDBClient(url=url, token=token, org=org,  timeout=900_000)

    # write_api = client.write_api(write_options=SYNCHRONOUS)
    query_api = client.query_api()

    data_frame = query_api.query_data_frame('from(bucket: "gs") '
                                        ' |> range(start: '+str(start_time)+', stop:'+str(end_time)+') '
                                        ' |> filter(fn: (r) => r._measurement == "'+measurement+'") '
                                        ' |> group(columns: ["_measurement", "_field"], mode: "by") '
                                        ' |> pivot(rowKey:["_time"], columnKey: ["_field"], valueColumn: "_value")')
    client.close()

    # df = data_frame.drop(columns=['result', 'table','_start', '_stop', '_measurement', 'topic', 'receivedFrom']).sort_values(by=["_time"]).reset_index(drop=True)
    data_frame.reset_index(inplace=True)
    df = data_frame[['_time', grouping_key]].sort_values(by=["_time"]).reset_index(drop=True)
    df["_time"] = pd.to_datetime(df["_time"])

    return df

In [7]:
reference = from_influx(url, token, org, "deliverMessage", start_reference, end_reference, "messageID")

reference.head(10)

,_time,messageID
0,2023-08-25 17:04:39.546325+00:00,ACQIARIg0XIChXgZjqo27UsUVxcaK0AdGwwYEXyhHK67Yq...
1,2023-08-25 17:04:39.546404+00:00,ACQIARIg0XIChXgZjqo27UsUVxcaK0AdGwwYEXyhHK67Yq...
2,2023-08-25 17:04:39.546432+00:00,ACQIARIg0XIChXgZjqo27UsUVxcaK0AdGwwYEXyhHK67Yq...
3,2023-08-25 17:04:39.546544+00:00,ACQIARIg0XIChXgZjqo27UsUVxcaK0AdGwwYEXyhHK67Yq...
4,2023-08-25 17:04:39.546587+00:00,ACQIARIg0XIChXgZjqo27UsUVxcaK0AdGwwYEXyhHK67Yq...
5,2023-08-25 17:04:39.546598+00:00,ACQIARIg0XIChXgZjqo27UsUVxcaK0AdGwwYEXyhHK67Yq...
6,2023-08-25 17:04:39.546609+00:00,ACQIARIg0XIChXgZjqo27UsUVxcaK0AdGwwYEXyhHK67Yq...
7,2023-08-25 17:04:39.546712+00:00,ACQIARIg0XIChXgZjqo27UsUVxcaK0AdGwwYEXyhHK67Yq...
8,2023-08-25 17:04:39.546713+00:00,ACQIARIg0XIChXgZjqo27UsUVxcaK0AdGwwYEXyhHK67Yq...
9,2023-08-25 17:04:39.546744+00:00,ACQIARIg0XIChXgZjqo27UsUVxcaK0AdGwwYEXyhHK67Yq...


In [8]:
par = experiments.loc[experiments["parameter"] == "d"]
start_query = par["startUnix"].min().astype(int)
end_query = par["endUnix"].max().astype(int)

print(start_query,end_query)

1693002357 1693020268


In [9]:
traces = from_influx(url, token, org, "deliverMessage", start_query, end_query, "messageID")
traces.head(10)

,_time,messageID
0,2023-08-25 22:25:57.053491+00:00,ACQIARIgqtBzkLlcGE0XEpfVekpvvEWamsRXGjEL1K5+bI...
1,2023-08-25 22:25:57.054470+00:00,ACQIARIgqtBzkLlcGE0XEpfVekpvvEWamsRXGjEL1K5+bI...
2,2023-08-25 22:25:57.054689+00:00,ACQIARIgqtBzkLlcGE0XEpfVekpvvEWamsRXGjEL1K5+bI...
3,2023-08-25 22:25:57.054913+00:00,ACQIARIgqtBzkLlcGE0XEpfVekpvvEWamsRXGjEL1K5+bI...
4,2023-08-25 22:25:57.055042+00:00,ACQIARIgqtBzkLlcGE0XEpfVekpvvEWamsRXGjEL1K5+bI...
5,2023-08-25 22:25:57.055332+00:00,ACQIARIgqtBzkLlcGE0XEpfVekpvvEWamsRXGjEL1K5+bI...
6,2023-08-25 22:25:57.056167+00:00,ACQIARIgqtBzkLlcGE0XEpfVekpvvEWamsRXGjEL1K5+bI...
7,2023-08-25 22:25:57.056831+00:00,ACQIARIgqtBzkLlcGE0XEpfVekpvvEWamsRXGjEL1K5+bI...
8,2023-08-25 22:25:57.057356+00:00,ACQIARIgqtBzkLlcGE0XEpfVekpvvEWamsRXGjEL1K5+bI...
9,2023-08-25 22:25:57.057869+00:00,ACQIARIgqtBzkLlcGE0XEpfVekpvvEWamsRXGjEL1K5+bI...


In [10]:
exp = experiments.loc[experiments['topology'] == "unl"]
exp = exp.loc[exp['parameter'] == "d"]
exp = pd.concat([exp, ref])

exp.head(25)

,experiment,start,end,topology,parameter,d,dlo,dhi,dscore,dlazy,dout,gossipFactor,interval,startUnix,endUnix
12,162,2023-08-25 22:25:57.7962343,2023-08-25 22:55:57.7997383,unl,d,24,6,12,4,8,2,0.25,1.0,1693002357,1693004157
13,163,2023-08-25 22:59:31.3828684,2023-08-25 23:29:31.3847606,unl,d,24,6,12,4,8,2,0.25,1.0,1693004371,1693006171
14,164,2023-08-25 23:33:04.8529178,2023-08-26 00:03:04.8573811,unl,d,24,6,12,4,8,2,0.25,1.0,1693006384,1693008184
15,165,2023-08-26 00:06:38.0702672,2023-08-26 00:36:38.0732183,unl,d,12,6,12,4,8,2,0.25,1.0,1693008398,1693010198
16,166,2023-08-26 00:40:12.5972314,2023-08-26 01:10:12.6028998,unl,d,12,6,12,4,8,2,0.25,1.0,1693010412,1693012212
17,167,2023-08-26 01:13:46.4289692,2023-08-26 01:43:46.4323589,unl,d,12,6,12,4,8,2,0.25,1.0,1693012426,1693014226
18,168,2023-08-26 01:47:19.9165818,2023-08-26 02:17:19.9201242,unl,d,6,6,12,4,8,2,0.25,1.0,1693014439,1693016239
19,169,2023-08-26 02:20:54.2917213,2023-08-26 02:50:54.2928258,unl,d,6,6,12,4,8,2,0.25,1.0,1693016454,1693018254
20,170,2023-08-26 02:54:28.9952952,2023-08-26 03:24:28.9976530,unl,d,6,6,12,4,8,2,0.25,1.0,1693018468,1693020268
0,150,2023-08-25 15:43:16.3243323,2023-08-25 16:13:16.3255309,unl,reference,8,6,12,4,8,2,0.25,1.0,1692978196,1692979996


In [11]:
traces = pd.concat([reference, traces])

traces.head(100)

,_time,messageID
0,2023-08-25 17:04:39.546325+00:00,ACQIARIg0XIChXgZjqo27UsUVxcaK0AdGwwYEXyhHK67Yq...
1,2023-08-25 17:04:39.546404+00:00,ACQIARIg0XIChXgZjqo27UsUVxcaK0AdGwwYEXyhHK67Yq...
2,2023-08-25 17:04:39.546432+00:00,ACQIARIg0XIChXgZjqo27UsUVxcaK0AdGwwYEXyhHK67Yq...
3,2023-08-25 17:04:39.546544+00:00,ACQIARIg0XIChXgZjqo27UsUVxcaK0AdGwwYEXyhHK67Yq...
4,2023-08-25 17:04:39.546587+00:00,ACQIARIg0XIChXgZjqo27UsUVxcaK0AdGwwYEXyhHK67Yq...
5,2023-08-25 17:04:39.546598+00:00,ACQIARIg0XIChXgZjqo27UsUVxcaK0AdGwwYEXyhHK67Yq...
6,2023-08-25 17:04:39.546609+00:00,ACQIARIg0XIChXgZjqo27UsUVxcaK0AdGwwYEXyhHK67Yq...
7,2023-08-25 17:04:39.546712+00:00,ACQIARIg0XIChXgZjqo27UsUVxcaK0AdGwwYEXyhHK67Yq...
8,2023-08-25 17:04:39.546713+00:00,ACQIARIg0XIChXgZjqo27UsUVxcaK0AdGwwYEXyhHK67Yq...
9,2023-08-25 17:04:39.546744+00:00,ACQIARIg0XIChXgZjqo27UsUVxcaK0AdGwwYEXyhHK67Yq...


In [12]:
traces.tail(100)

,_time,messageID
1611306,2023-08-26 03:24:17.047822+00:00,ACQIARIgFEGM3FOQ2H79LXNaPHJ5b+cZHWLYTjh+h8ID3c...
1611307,2023-08-26 03:24:17.048289+00:00,ACQIARIgFEGM3FOQ2H79LXNaPHJ5b+cZHWLYTjh+h8ID3c...
1611308,2023-08-26 03:24:17.115940+00:00,ACQIARIgo+G6XPg1Kjdwpc5M7CxrGlQXg5se6bHlFaYJjY...
1611309,2023-08-26 03:24:17.116586+00:00,ACQIARIgo+G6XPg1Kjdwpc5M7CxrGlQXg5se6bHlFaYJjY...
1611310,2023-08-26 03:24:17.131257+00:00,ACQIARIgFcrOE6zggqC78TeZqH1n+PmFRXIUXDO+uNuTnL...
1611311,2023-08-26 03:24:17.131521+00:00,ACQIARIgFcrOE6zggqC78TeZqH1n+PmFRXIUXDO+uNuTnL...
1611312,2023-08-26 03:24:17.157461+00:00,ACQIARIg2E5YMFznHacBOytrj+HcCNQBtpgMqFDEHtnRlQ...
1611313,2023-08-26 03:24:17.157855+00:00,ACQIARIg2E5YMFznHacBOytrj+HcCNQBtpgMqFDEHtnRlQ...
1611314,2023-08-26 03:24:17.191099+00:00,ACQIARIgBeo1PFEVY0FfdOnl91TUUgiSekNH9/+KNZKwGV...
1611315,2023-08-26 03:24:17.195088+00:00,ACQIARIgBeo1PFEVY0FfdOnl91TUUgiSekNH9/+KNZKwGV...


In [13]:
# def group_time(df, expRaw, parameter,grouping_key, start, end):
# gb = group_time(traces, exp, graph['parameter'], graph['grouping_key'], start_reference, end_query)

df=traces
expRaw=exp
start = start_reference
end = end_query
grouping_key = "messageID"
parameter = "d"

expTime = expRaw.loc[expRaw['startUnix']>= int(start)].loc[expRaw['endUnix'] <= int(end)]
expTime.head(20)

,experiment,start,end,topology,parameter,d,dlo,dhi,dscore,dlazy,dout,gossipFactor,interval,startUnix,endUnix
12,162,2023-08-25 22:25:57.7962343,2023-08-25 22:55:57.7997383,unl,d,24,6,12,4,8,2,0.25,1.0,1693002357,1693004157
13,163,2023-08-25 22:59:31.3828684,2023-08-25 23:29:31.3847606,unl,d,24,6,12,4,8,2,0.25,1.0,1693004371,1693006171
14,164,2023-08-25 23:33:04.8529178,2023-08-26 00:03:04.8573811,unl,d,24,6,12,4,8,2,0.25,1.0,1693006384,1693008184
15,165,2023-08-26 00:06:38.0702672,2023-08-26 00:36:38.0732183,unl,d,12,6,12,4,8,2,0.25,1.0,1693008398,1693010198
16,166,2023-08-26 00:40:12.5972314,2023-08-26 01:10:12.6028998,unl,d,12,6,12,4,8,2,0.25,1.0,1693010412,1693012212
17,167,2023-08-26 01:13:46.4289692,2023-08-26 01:43:46.4323589,unl,d,12,6,12,4,8,2,0.25,1.0,1693012426,1693014226
18,168,2023-08-26 01:47:19.9165818,2023-08-26 02:17:19.9201242,unl,d,6,6,12,4,8,2,0.25,1.0,1693014439,1693016239
19,169,2023-08-26 02:20:54.2917213,2023-08-26 02:50:54.2928258,unl,d,6,6,12,4,8,2,0.25,1.0,1693016454,1693018254
20,170,2023-08-26 02:54:28.9952952,2023-08-26 03:24:28.9976530,unl,d,6,6,12,4,8,2,0.25,1.0,1693018468,1693020268
0,150,2023-08-25 15:43:16.3243323,2023-08-25 16:13:16.3255309,unl,reference,8,6,12,4,8,2,0.25,1.0,1692978196,1692979996


In [14]:
 #Make the db in memory
conn = sqlite3.connect(':memory:')
#write the tables
df.to_sql('df', conn, index=False)
expTime.to_sql('expTime', conn, index=False)

qry = '''
        select  
            df._time,
            expTime.start as min,
            expTime.end as max,
            df.'''+grouping_key+''',
            expTime.experiment,
            expTime.'''+parameter+'''
        from
            df join expTime on
            df._time between expTime.start and expTime.end
        '''
dfNew = pd.read_sql_query(qry, conn)
# print(dfNew)

dfNew = dfNew.set_index('experiment')#.rename(columns={"_time": "min"})#.drop(columns=["messageID"])

#dfNew['min'] = 
dfNew['min'] = pd.to_datetime(dfNew["min"], format='mixed')
dfNew['max'] = pd.to_datetime(dfNew["max"], format='mixed')
dfNew['_time'] = pd.to_datetime(dfNew["_time"], format='mixed')
# dfNew['_min'] = pd.to_datetime(dfNew["_min"])

dfNew.head(100)

,_time,min,max,messageID,d
experiment,,,,,
152,2023-08-25 17:04:39.546325+00:00,2023-08-25 16:50:23.689999200,2023-08-25 17:20:23.692199100,ACQIARIg0XIChXgZjqo27UsUVxcaK0AdGwwYEXyhHK67Yq...,8
152,2023-08-25 17:04:39.546404+00:00,2023-08-25 16:50:23.689999200,2023-08-25 17:20:23.692199100,ACQIARIg0XIChXgZjqo27UsUVxcaK0AdGwwYEXyhHK67Yq...,8
152,2023-08-25 17:04:39.546432+00:00,2023-08-25 16:50:23.689999200,2023-08-25 17:20:23.692199100,ACQIARIg0XIChXgZjqo27UsUVxcaK0AdGwwYEXyhHK67Yq...,8
152,2023-08-25 17:04:39.546544+00:00,2023-08-25 16:50:23.689999200,2023-08-25 17:20:23.692199100,ACQIARIg0XIChXgZjqo27UsUVxcaK0AdGwwYEXyhHK67Yq...,8
152,2023-08-25 17:04:39.546587+00:00,2023-08-25 16:50:23.689999200,2023-08-25 17:20:23.692199100,ACQIARIg0XIChXgZjqo27UsUVxcaK0AdGwwYEXyhHK67Yq...,8
152,2023-08-25 17:04:39.546598+00:00,2023-08-25 16:50:23.689999200,2023-08-25 17:20:23.692199100,ACQIARIg0XIChXgZjqo27UsUVxcaK0AdGwwYEXyhHK67Yq...,8
152,2023-08-25 17:04:39.546609+00:00,2023-08-25 16:50:23.689999200,2023-08-25 17:20:23.692199100,ACQIARIg0XIChXgZjqo27UsUVxcaK0AdGwwYEXyhHK67Yq...,8
152,2023-08-25 17:04:39.546712+00:00,2023-08-25 16:50:23.689999200,2023-08-25 17:20:23.692199100,ACQIARIg0XIChXgZjqo27UsUVxcaK0AdGwwYEXyhHK67Yq...,8
152,2023-08-25 17:04:39.546713+00:00,2023-08-25 16:50:23.689999200,2023-08-25 17:20:23.692199100,ACQIARIg0XIChXgZjqo27UsUVxcaK0AdGwwYEXyhHK67Yq...,8


In [19]:
def insert_missing_time(dfAggTime, parameter):
    # Fill the voids
    # minTime = dfAggTime.groupby(['experiment']).agg('min').drop(columns=['count'])
    # maxTime = dfAggTime.groupby(['experiment']).agg('max').drop(columns=['count'])

    # minTime = dfAggTime.drop(columns=['count','max','_time']).drop_duplicates()
    # minTime = dfAggTime.drop(columns=['count','min','_time']).drop_duplicates()    
    # maxTime.head(10)
    # print(maxTime)

    # minMax = minTime.merge(maxTime, on=['experiment']).rename(columns={"min_x": "min", "min_y": "max", parameter+"_x": parameter}).reset_index()
    # # minMax.head(10)

    date_list = pd.date_range(dfAggTime['min'].min(), dfAggTime['max'].max(),freq='1s',tz=None)

    dates = pd.DataFrame(date_list).rename(columns={0:"_time"})
    dates['count'] = 0
    # print(dates)

    dates['_time'] = pd.to_datetime(dates["_time"], format='mixed')#.tz_localize(None)
    dfAggTime['_time'] = pd.to_datetime(dfAggTime["_time"], format='mixed')#.tz_localize(None)
    dfAggTime['min'] = pd.to_datetime(dfAggTime["min"], format='mixed')#.tz_localize(None)
    dfAggTime['max'] = pd.to_datetime(dfAggTime["max"], format='mixed')#.tz_localize(None)

    dates["_time"] = dates["_time"].dt.tz_localize(None)
    dfAggTime["_time"] = dfAggTime["_time"].dt.tz_localize(None)
    dfAggTime["min"] = dfAggTime["min"].dt.tz_localize(None)
    dfAggTime["max"] = dfAggTime["max"].dt.tz_localize(None)

    #Make the db in memory
    conn = sqlite3.connect(':memory:')
    #write the tables
    dfAggTime.to_sql('minMax', conn, index=False)
    dates.to_sql('dates', conn, index=False)

    qry = '''
        select distinct
            dates._time as _time,
            minMax.min,
            minMax.max,
            minMax.'''+parameter+''',
            minMax.experiment,
            dates.count
        from
            dates join minMax on
            dates._time between minMax.min and minMax.max
        '''
    dfFill = pd.read_sql_query(qry, conn)
    # dfFill.head(10)

    dfFill['_time'] = pd.to_datetime(dfFill["_time"], format='mixed')#.tz_localize(None)
    dfAggTime['_time'] = pd.to_datetime(dfAggTime["_time"], format='mixed')#.tz_localize(None)

    dfFill["_time"] = dfFill["_time"].dt.tz_localize(None)
    dfAggTime["_time"] = dfAggTime["_time"].dt.tz_localize(None)
    
    #write the tables
    dfFill.to_sql('fill', conn, index=False)
    dfAggTime.to_sql('df', conn, index=False)

    qry = '''
        select distinct
           experiment,
           '''+parameter+''',
           _time,
           count
        from fill
        where fill._time not in (SELECT DISTINCT _time FROM df)
        '''
    dfMissingTime = pd.read_sql_query(qry, conn).reset_index(drop=True).drop_duplicates()

    df =  pd.concat([dfMissingTime.reset_index(drop=True), dfAggTime.drop(columns=['min','max']).reset_index(drop=True)])#.sort_values(by=['_time'])
    df["_time"] = pd.to_datetime(df["_time"], format='mixed')
    df = df.sort_values(by=['_time']).drop_duplicates()

    return df


In [16]:
#Try resampling for every 5 seconds
dfNoIndex = dfNew.reset_index()
# dfNoIndex.head(10)

by_time = dfNoIndex.groupby([dfNoIndex['experiment'],dfNoIndex[parameter],dfNoIndex["min"],dfNoIndex["max"],pd.Grouper(key="_time", freq='1s')])[grouping_key].count().reset_index()
dfAggTime = by_time.rename(columns={grouping_key: "count"})

dfAggTime = insert_missing_time(dfAggTime, parameter)
dfNoIndex = dfAggTime.reset_index()
# dfNoIndex.head(10)

#Agregate to the time frame we want
by_time2 = dfNoIndex.groupby([dfNoIndex['experiment'],dfNoIndex[parameter],pd.Grouper(key="_time", freq='10s')])['count'].sum().reset_index()
dfAggTime = by_time2#.rename(columns={grouping_key: "count"})

# dfAggTime.head(100)

#Min datetime of each experiment
minTime = dfAggTime.groupby(['experiment']).agg('min').drop(columns=[parameter, 'count'])

minTime.head(20)

,_time
experiment,
152,2023-08-25 16:50:20
162,2023-08-25 22:25:50
163,2023-08-25 22:59:30
164,2023-08-25 23:33:00
165,2023-08-26 00:06:30
166,2023-08-26 00:40:10
167,2023-08-26 01:13:40
168,2023-08-26 01:47:20
169,2023-08-26 02:20:50


In [17]:
#Join to calculate delta
dfWithMin = dfAggTime.merge(minTime, on='experiment', how='left').rename(columns={'_time_x': '_time', '_time_y': '_min'}).drop_duplicates()

dfWithMin['_time'] = pd.to_datetime(dfWithMin["_time"], format='mixed')
dfWithMin['_min'] = pd.to_datetime(dfWithMin["_min"],  format='mixed')

# # #Calculate delta in seconds 
dfWithMin["delta"] = ((dfWithMin["_time"] - dfWithMin["_min"]) / pd.Timedelta(seconds=1)).astype(int)
dfWithMin.head(10)

,experiment,d,_time,count,_min,delta
0,152,8,2023-08-25 16:50:20,0,2023-08-25 16:50:20,0
1,152,8,2023-08-25 16:50:30,0,2023-08-25 16:50:20,10
2,152,8,2023-08-25 16:50:40,0,2023-08-25 16:50:20,20
3,152,8,2023-08-25 16:50:50,0,2023-08-25 16:50:20,30
4,152,8,2023-08-25 16:51:00,0,2023-08-25 16:50:20,40
5,152,8,2023-08-25 16:51:10,0,2023-08-25 16:50:20,50
6,152,8,2023-08-25 16:51:20,0,2023-08-25 16:50:20,60
7,152,8,2023-08-25 16:51:30,0,2023-08-25 16:50:20,70
8,152,8,2023-08-25 16:51:40,0,2023-08-25 16:50:20,80
9,152,8,2023-08-25 16:51:50,0,2023-08-25 16:50:20,90


In [18]:
#Aggregate by time
gb = dfWithMin.groupby(['delta','experiment', parameter])['count'].agg(["sum"]).sort_values(by=["experiment", "delta"])
# gb.head(100)

#Average by interval
intv = gb.groupby([parameter,'delta']).agg(["mean"]).sort_values(by=[parameter, "delta"])
intv.columns = intv.columns.droplevel(0)#.droplevel(1)
# intv.reset_index(level=0, inplace=True)
intv.reset_index(inplace=True)

intv.head(10)


,d,delta,mean
0,6,0,93.666667
1,6,10,119.000000
2,6,20,108.333333
3,6,30,134.333333
4,6,40,143.000000
5,6,50,126.666667
6,6,60,139.333333
7,6,70,139.666667
8,6,80,128.333333
9,6,90,142.000000
